In [72]:
import pandas as pd 
#Reading user file:
u_cols =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,
 encoding='latin-1')

n_users = users.shape[0]
print('Number of users:', users)

Number of users:      user_id  age sex     occupation zip_code
0          1   24   M     technician    85711
1          2   53   F          other    94043
2          3   23   M         writer    32067
3          4   24   M     technician    43537
4          5   33   F          other    15213
..       ...  ...  ..            ...      ...
938      939   26   F        student    33319
939      940   32   M  administrator    02215
940      941   20   M        student    97229
941      942   48   F      librarian    78209
942      943   22   M        student    77841

[943 rows x 5 columns]


In [73]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.values
rate_test = ratings_test.values

print(ratings_base)


       user_id  movie_id  rating  unix_timestamp
0            1         1       5       874965758
1            1         2       3       876893171
2            1         3       4       878542960
3            1         4       3       876893119
4            1         5       3       889751712
...        ...       ...     ...             ...
90565      943      1047       2       875502146
90566      943      1074       4       888640250
90567      943      1188       3       888640250
90568      943      1228       3       888640275
90569      943      1330       3       888692465

[90570 rows x 4 columns]


In [74]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,
 encoding='latin-1')

n_items = items.shape[0]
# print ('Number of items:', items)
items.tail(5)


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1681,1682,Scream of Stone (Schrei aus Stein) (1991),08-Mar-1996,NaN,http://us.imdb.com/M/title-exact?Schrei%20aus%...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
X0 = items.values
X_train_counts = X0[:, -19:]
print(X_train_counts)

[[0 0 0 ... 0 0 0]
 [0 1 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [84]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()
print(tfidf)

[[0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.54 0.65 ... 0.54 0.   0.  ]
 [0.   0.   0.   ... 1.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


In [96]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    in each line of rate_matrix, we have infor: user_id, item_id, rating (scores), time_stamp
    we care about the first three values
    return (item_ids, scores) rated by user user_id
    """
    y = rate_matrix[:,0] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1 
    # while index in python starts from 0
    ids = np.where(y == user_id +1)[0] 
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0 
    scores = rate_matrix[ids, 2]

    return (item_ids, scores)


In [98]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = tfidf.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):    
    ids, scores = get_items_rated_by_user(rate_train, n)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = tfidf[ids, :]
    clf.fit(Xhat, scores)

    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  20  21  22  23  24  25  26  27  28  29  30  31  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 117 118 119 120 121 122 123 124 125 126 127 128 129
 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147
 148 149 150 151 152 153 155 156 157 158 160 161 162 163 164 165 166 167
 168 169 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186
 187 189 190 191 192 193 194 195 196 197 198 199 200 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 25

In [79]:
print(tfidf)
Yhat = tfidf.dot(W) + b
print(Yhat)

[[0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.54 0.65 ... 0.54 0.   0.  ]
 [0.   0.   0.   ... 1.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]
[[2.92 4.   1.9  ... 5.04 4.78 3.17]
 [2.8  3.44 3.3  ... 1.1  4.76 3.7 ]
 [3.48 1.56 1.27 ... 7.36 4.19 3.81]
 ...
 [4.13 4.1  3.02 ... 9.54 4.64 3.27]
 [3.57 3.45 2.93 ... 3.68 3.66 2.76]
 [4.13 3.8  3.1  ... 9.54 4.42 3.93]]


In [100]:
n = 10
np.set_printoptions(precision=2) # 2 digits after . 
ids, scores = get_items_rated_by_user(rate_train, n)
# print(Yhat[n, ids])
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', Yhat[ids, n])

[  7   8  10  11  14  21  23  24  27  28  38  39  41  46  50  51  53  55
  56  57  68  69  78  82  85  87  89  93  96  97  99 106 108 119 120 122
 124 134 167 172 174 175 179 184 189 190 193 195 202 203 207 210 212 214
 215 221 227 228 229 236 237 238 240 257 258 259 267 273 276 285 289 290
 299 300 311 316 317 323 331 349 355 356 364 366 369 371 381 382 385 392
 394 398 400 401 404 413 422 426 427 428 429 430 432 433 434 448 450 454
 503 507 516 520 523 525 526 543 548 560 572 576 579 596 602 645 651 653
 658 659 661 662 689 691 698 706 709 712 713 714 715 716 717 718 719 720
 721 723 725 726 727 728 729 730 732 733 734 735 736 737 738 740 741 742
 743 744 745 746 747 748 749 750 751]
Rated movies ids : [  7   8  10  11  14  21  23  24  27  28  38  39  41  46  50  51  53  55
  56  57  68  69  78  82  85  87  89  93  96  97  99 106 108 119 120 122
 124 134 167 172 174 175 179 184 189 190 193 195 202 203 207 210 212 214
 215 221 227 228 229 236 237 238 240 257 258 259 267 273 276 285 28